In [12]:
!pip install --upgrade google-cloud-storage transformers accelerate torch torchvision matplotlib
!pip install easyocr
!pip install pytesseract
!apt-get update
!apt-get install -y tesseract-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.9/174.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15

  Using cached nvidia_cusparse_cu12-12.5.4.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cusparse_cu12-12.5.4.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (216.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.1 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.il

In [20]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/valid-might-460212-k2-aabca356a049.json'

import tempfile
from google.cloud import storage
import pandas as pd

GCS_BUCKET = 'de-project2-bucket-1'
GCS_XLSX_PREFIX = 'kyobo/excel'
GCS_CSV_PREFIX = 'kyobo/csv'

client = storage.Client()
bucket = client.bucket(GCS_BUCKET)

# Colab에서 GCS 파일을 로컬로 다운로드
!gsutil cp gs://de-project2-bucket-1/kyobo/csv/combined/after_preprocessing.csv /content/meta_data.csv

Copying gs://de-project2-bucket-1/kyobo/csv/combined/after_preprocessing.csv...
\ [1 files][ 56.8 MiB/ 56.8 MiB]                                                
Operation completed over 1 objects/56.8 MiB.                                     


In [2]:
import pandas as pd

# 파일 읽기
df = pd.read_csv('/content/meta_data.csv', encoding='utf-8-sig')
df.head()

,index,product_code,product_id,title,list_price,sale_price,discount_rate,point_rate,expected_points,author,publisher,pub_date,category,image_url
0,1,9.788998e+12,S000001632467,모순,"13,000","11,700",10%,5%,650,양귀자,쓰다,20130401.0,소설,https://storage.googleapis.com/de-project2-buc...
1,2,9.791162e+12,S000001808685,파과,"15,800","14,220",10%,5%,790,구병모,위즈덤하우스,20180416.0,소설,https://storage.googleapis.com/de-project2-buc...
2,3,9.788936e+12,S000000610612,소년이 온다,"15,000","13,500",10%,5%,750,한강,창비,20140519.0,소설,https://storage.googleapis.com/de-project2-buc...
3,4,9.788958e+12,S000216345011,포기할 자유,"18,500","16,650",10%,5%,920,이재구,아마존북스,20250421.0,소설,https://storage.googleapis.com/de-project2-buc...
4,5,9.788936e+12,S000216111714,혼모노,"18,000","16,200",10%,5%,900,성해나,창비,20250328.0,소설,https://storage.googleapis.com/de-project2-buc...


In [3]:
image_urls = df['image_url'].dropna().tolist()
product_ids = df['product_id'].tolist()
print(f"✅ 총 {len(image_urls)}개의 이미지 URL을 불러왔습니다.")

✅ 총 253302개의 이미지 URL을 불러왔습니다.


In [11]:
!pip install easyocr langdetect

  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached easyocr-1.7.2-py3-none-any.whl (2.9 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl (207.5 MB)
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.3.0.75
    Uninstalling nvidia-cudnn-cu12-9.3.0.75:
      Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling 

In [ ]:
# 병렬화
# ✅ 필요한 패키지 임포트
import gspread
from google.auth import default
from google.colab import auth
import re
import easyocr
from PIL import Image
import requests
from io import BytesIO
import numpy as np
from langdetect import detect
from tqdm import tqdm
from collections import defaultdict
import csv
from concurrent.futures import ThreadPoolExecutor

# ✅ EasyOCR 초기화 (Colab GPU 있을 경우 gpu=True 권장)
reader = easyocr.Reader(['ko', 'en'], gpu=True)

# ✅ OCR 처리 함수 (한 이미지 처리)
def process_image(pid_url):
    pid, url = pid_url
    result = {
        'pid': pid,
        'url': url,
        'texts': [],
        'langs': [],
        'widths': [],
        'heights': [],
        'area_ratios': []
    }
    try:
        response = requests.get(url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        width, height = image.size
        ocr_result = reader.readtext(np.array(image))

        if not ocr_result:
            return result

        for (bbox, text, conf) in ocr_result:
            if len(text.strip()) == 0:
                continue
            try:
                lang = detect(text)
            except:
                lang = "unknown"

            x_coords = [point[0] for point in bbox]
            y_coords = [point[1] for point in bbox]
            x_min, x_max = min(x_coords), max(x_coords)
            y_min, y_max = min(y_coords), max(y_coords)
            box_width = x_max - x_min
            box_height = y_max - y_min
            area_ratio = (box_width * box_height) / (width * height)

            result['texts'].append(text)
            result['langs'].append(lang)
            result['widths'].append(round(box_width, 1))
            result['heights'].append(round(box_height, 1))
            result['area_ratios'].append(round(area_ratio, 4))

    except Exception as e:
        print(f"❌ {pid} 처리 실패: {e}")
        result['texts'].append('error')
        result['langs'].append('')
        result['widths'].append('')
        result['heights'].append('')
        result['area_ratios'].append('')
    return result

# ✅ 병렬 처리 실행
with ThreadPoolExecutor(max_workers=8) as executor:  # 코어에 맞게 max_workers 조절
    results = list(tqdm(executor.map(process_image, zip(product_ids, image_urls)), total=len(image_urls)))

# ✅ 결과를 딕셔너리 형태로 변환 (기존 코드와 호환 위해)
ocr_dict = defaultdict(lambda: {'url': '', 'texts': [], 'langs': [], 'widths': [], 'heights': [], 'area_ratios': []})
for res in results:
    pid = res['pid']
    ocr_dict[pid]['url'] = res['url']
    ocr_dict[pid]['texts'] = res['texts']
    ocr_dict[pid]['langs'] = res['langs']
    ocr_dict[pid]['widths'] = res['widths']
    ocr_dict[pid]['heights'] = res['heights']
    ocr_dict[pid]['area_ratios'] = res['area_ratios']

# ✅ 리스트들을 문자열로 변환해서 한 행으로 구성
results_aggregated = []
for pid, info in ocr_dict.items():
    results_aggregated.append([
        pid,
        info['url'],
        ", ".join(info['texts']),
        ", ".join(info['langs']),
        ", ".join(map(str, info['widths'])),
        ", ".join(map(str, info['heights'])),
        ", ".join(map(str, info['area_ratios']))
    ])

# ✅ CSV 저장 경로 (Colab 로컬)
csv_path = "/content/ocr_results_aggregated.csv"

# ✅ CSV 파일로 저장
with open(csv_path, "w", newline='', encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["판매상품ID", "Image URL", "텍스트(리스트)", "언어(리스트)", "텍스트 너비(리스트)", "텍스트 높이(리스트)", "이미지 내 면적 비율(리스트)"])
    writer.writerows(results_aggregated)

print(f"✅ Colab에 CSV 저장 완료: {csv_path}")

# ✅ GCS 업로드 경로 (원하는 경로로 수정 가능)
gcs_path = "gs://de-project2-bucket-1/kyobo/csv/ocr_results_aggregated.csv"

# ✅ GCS에 업로드
!gsutil cp {csv_path} {gcs_path}

print(f"✅ GCS 업로드 완료: {gcs_path}")


  0%|          | 178/253302 [00:32<10:43:19,  6.56it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(
 29%|██▉       | 72902/253302 [1:48:54<4:05:16, 12.26it/s]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:760: UserWarning: Metadata Warning, tag 282 had too many entries: 4, expected 1
  warnings.warn(
100%|██████████| 253302/253302 [6:31:39<00:00, 10.78it/s]


✅ Colab에 CSV 저장 완료: /content/ocr_results_aggregated.csv
Copying file:///content/ocr_results_aggregated.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/81.2 MiB.                                     
✅ GCS 업로드 완료: gs://de-project2-bucket-1/kyobo/csv/ocr_results_aggregated.csv


In [ ]:
# 기존 코드 (느림)
# ✅ 필요한 패키지 임포트
import gspread
from google.auth import default
from google.colab import auth
import re
import easyocr
from PIL import Image
import requests
from io import BytesIO
import numpy as np
from langdetect import detect
from tqdm import tqdm
from collections import defaultdict
import csv

# ✅ EasyOCR 초기화
reader = easyocr.Reader(['ko', 'en'])

# ✅ OCR 결과 저장용 딕셔너리
ocr_dict = defaultdict(lambda: {'url': '', 'texts': [], 'langs': [], 'widths': [], 'heights': [], 'area_ratios': []})

# ✅ OCR 실행 및 딕셔너리에 통합 저장
for pid, url in tqdm(zip(product_ids, image_urls), total=len(image_urls)):
    try:
        response = requests.get(url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        width, height = image.size
        ocr_result = reader.readtext(np.array(image))

        ocr_dict[pid]['url'] = url

        if not ocr_result:
            continue

        for (bbox, text, conf) in ocr_result:
            if len(text.strip()) == 0:
                continue
            try:
                lang = detect(text)
            except:
                lang = "unknown"

            x_coords = [point[0] for point in bbox]
            y_coords = [point[1] for point in bbox]
            x_min, x_max = min(x_coords), max(x_coords)
            y_min, y_max = min(y_coords), max(y_coords)
            box_width = x_max - x_min
            box_height = y_max - y_min
            area_ratio = (box_width * box_height) / (width * height)

            ocr_dict[pid]['texts'].append(text)
            ocr_dict[pid]['langs'].append(lang)
            ocr_dict[pid]['widths'].append(round(box_width, 1))
            ocr_dict[pid]['heights'].append(round(box_height, 1))
            ocr_dict[pid]['area_ratios'].append(round(area_ratio, 4))

    except Exception as e:
        print(f"❌ {pid} 처리 실패: {e}")
        ocr_dict[pid]['url'] = url
        ocr_dict[pid]['texts'].append('error')
        ocr_dict[pid]['langs'].append('')
        ocr_dict[pid]['widths'].append('')
        ocr_dict[pid]['heights'].append('')
        ocr_dict[pid]['area_ratios'].append('')

# ✅ 리스트들을 문자열로 변환해서 한 행으로 구성
results_aggregated = []
for pid, info in ocr_dict.items():
    results_aggregated.append([
        pid,
        info['url'],
        ", ".join(info['texts']),
        ", ".join(info['langs']),
        ", ".join(map(str, info['widths'])),
        ", ".join(map(str, info['heights'])),
        ", ".join(map(str, info['area_ratios']))
    ])

# ✅ CSV 저장 경로 (Colab 로컬)
csv_path = "/content/ocr_results_aggregated.csv"

# ✅ CSV 파일로 저장
with open(csv_path, "w", newline='', encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["판매상품ID", "Image URL", "텍스트(리스트)", "언어(리스트)", "텍스트 너비(리스트)", "텍스트 높이(리스트)", "이미지 내 면적 비율(리스트)"])
    writer.writerows(results_aggregated)

print(f"✅ Colab에 CSV 저장 완료: {csv_path}")

# ✅ GCS 업로드 경로 (원하는 경로로 수정 가능)
gcs_path = "gs://de-project2-bucket-1/kyobo/csv/ocr_results_aggregated.csv"

# ✅ GCS에 업로드
!gsutil cp {csv_path} {gcs_path}

print(f"✅ GCS 업로드 완료: {gcs_path}")


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

  0%|          | 363/253302 [10:32<122:29:13,  1.74s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

def visualize_ocr_on_image(url, ocr_result):
    response = requests.get(url, timeout=10)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(image)

    for (bbox, text, conf) in ocr_result:
        if len(text.strip()) == 0:
            continue
        # bbox 좌표
        x_coords = [point[0] for point in bbox]
        y_coords = [point[1] for point in bbox]
        x_min, y_min = min(x_coords), min(y_coords)
        box_width = max(x_coords) - x_min
        box_height = max(y_coords) - y_min

        # 사각형 그리기
        rect = patches.Rectangle(
            (x_min, y_min), box_width, box_height,
            linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)
        # 텍스트 표시
        ax.text(x_min, y_min - 10, text, fontsize=12, color='red', weight='bold')

    plt.axis('off')
    plt.show()

# 예시: 처음 3개 결과 시각화 (실제 OCR 결과를 다시 얻어야 하므로, 아래처럼 처리)
for idx in range(min(20, len(image_urls))):
    url = image_urls[idx]
    print(f"=== 시각화: 판매상품ID={product_ids[idx]}, 이미지 URL={url} ===")
    response = requests.get(url, timeout=10)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    ocr_result = reader.readtext(np.array(image))
    visualize_ocr_on_image(url, ocr_result)


Output hidden; open in https://colab.research.google.com to view.

# google sheet

In [ ]:
import gspread
from google.auth.transport.requests import Request
from google.colab import auth
from google.auth import default
import re

import os
import io
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from google.cloud import storage
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import requests

# Google 계정 인증
auth.authenticate_user()

# 인증된 기본 자격증명 얻기
creds, _ = default()
gc = gspread.authorize(creds)

# 시트 열기
spreadsheet = gc.open("kyobo_metadata_sheet")
sheet = spreadsheet.worksheet("object_detection")

# 전체 데이터를 가져와서 O열 추출 (인덱스 14 = 15번째 열)
data = sheet.get_all_values()
o_column_cells = [row[14] for row in data[1:] if len(row) > 14]

# 수식에서 URL 추출
def extract_image_url_from_formula(cell):
    match = re.search(r'"(https://[^"]+)"', cell)
    return match.group(1) if match else None

image_urls = [extract_image_url_from_formula(cell) for cell in o_column_cells if extract_image_url_from_formula(cell)]

print(f"✅ 총 {len(image_urls)}개의 이미지 URL을 불러왔습니다.")
